## HW 4.3

### 1. Run ChIPQC for:
    https://drive.google.com/open?id=1dKLt_m_vnHjNVo-1qoACu9GZZC9OBa9v (Links to an external site.)Links to an external site.
    EFF_H3K4Me1_1_srt_dupr_chr11.bam
    EFF_H3K4Me1_2_srt_dupr_chr11.bam
    EFF_Input_srt_dupr_chr11.bam (Control)
- Sort and index bam file
- Run MACS2
- Create sample sheet
- ChIPQC

In [2]:
%%bash
cd ChIP-seq.HW4.3/
FILES=$(ls | grep ".bam")
for FILE in $FILES
do
    NAME=${FILE%%.*}.sorted.bam
    samtools sort $FILE -o $NAME
    samtools index $NAME
    
    macs2 callpeak \
    -t $NAME \
    -c EFF_Input_srt_dupr_chr11.bam \
    -f BAM \
    -n ${NAME}_MACS2 \
    -g mm \
    -p 1e-2 \
    --broad \
    --nomodel \
    --bdg \
    --shift 0 \
    --extsize 147 \
    --keep-dup all
done

INFO  @ Wed, 19 Dec 2018 15:30:44: 
# Command line: callpeak -t EFF_H3K4Me1_1_srt_dupr_chr11.sorted.bam -c EFF_Input_srt_dupr_chr11.bam -f BAM -n EFF_H3K4Me1_1_srt_dupr_chr11.sorted.bam_MACS2 -g mm -p 1e-2 --broad --nomodel --bdg --shift 0 --extsize 147 --keep-dup all
# ARGUMENTS LIST:
# name = EFF_H3K4Me1_1_srt_dupr_chr11.sorted.bam_MACS2
# format = BAM
# ChIP-seq file = ['EFF_H3K4Me1_1_srt_dupr_chr11.sorted.bam']
# control file = ['EFF_Input_srt_dupr_chr11.bam']
# effective genome size = 1.87e+09
# band width = 300
# model fold = [5, 50]
# pvalue cutoff for narrow/strong regions = 1.00e-02
# pvalue cutoff for broad/weak regions = 1.00e-01
# qvalue will not be calculated and reported as -1 in the final output.
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 1000 bps and 

#### 2.Use Homer to search for TF motifs in broadPeak file (from Q1, EFF_H3K4Me1_1_srt_dupr_chr11.bam)
genome reference: genome_chr11.fa

In [17]:
%%bash
source ~/.bash_profile
cd ~/Documents/Dev/SRCourse/original/Applied-Bioinformatics/Unit2-module2-ChIPseq/ChIP-seq.HW4.3/
KEYWORD="EFF_H3K4Me1_1_srt_dupr_chr11.sorted"
BED_NAME=$(ls | grep $KEYWORD | grep broadPeak)
CHR11_GENOME="genome_chr11.fa"
OUT_NAME=${KEYWORD}_HomerMF
ERR_NAME=${KEYWORD}_HomerMF.out

findMotifsGenome.pl $BED_NAME $CHR11_GENOME $OUT_NAME &> $ERR_NAME

##### What are the top 5 knownResults?

In [55]:
%%bash
cd ChIP-seq.HW4.3/EFF_H3K4Me1_1_srt_dupr_chr11.sorted_HomerMF/
RAW=$(head -6 knownResults.txt | cut -f 1 | sed '1d')
for name in $RAW
do
    echo ${name%%(*}
done

ETV1
Ets1-distal
Fli1
ERG
ETS1


##### What is the difference between knownResults and homerResults?
According to the [Manual](http://homer.ucsd.edu/homer/introduction/basics.html),  
*De novo motif discovery allows you to directly query the sequence to discover which motifs are the MOST enriched sequences in your target set.  Known motif discovery will simply tell you which of the known motifs is most enriched in your target set.*

The other thing is the reliability:  
*The greatest advantage to using known motifs is found when you have a limited set of target sequences.  The less data that is available or the weaker the true signal, it is difficult for de novo motif finding to accurately define a signal that is significant.  Known motifs have the advantage of many less degrees of freedom and in may cases find the correct motifs when the enrichment falls below the 1e-10 thresholds for reliability when considering de novo results.*